In [15]:
import joblib
import numpy as np
import pandas as pd
from copy import deepcopy

# Carregar os artefatos salvos do main.py
modelo, X_test, explicacoes, features_usadas_lista, X_train, y_test, class_names = joblib.load('artefatos_para_teste_pi.pkl')

#modelo, X_test, explicacoes, features_usadas_lista, X_train = joblib.load('artefatos_para_teste_pi.pkl')
y_test = modelo.predict(X_test)  # Isso vai gerar as classes preditas
#class_names = ['Classe 0', 'Classe 1']


# Função para perturbar as features que não estão na explicação
def perturbar_instancia(instancia, features_a_manter, X_train):
    perturbada = instancia.copy()
    for f in instancia.index:
        if f not in features_a_manter:
            # Sorteia um valor realista da feature, com base nos dados de treino
            perturbada[f] = np.random.choice(X_train[f])
    return perturbada

# Função para validar a robustez das PI-explicações
def testar_robustez_pi(modelo, X_test, explicacoes, features_usadas_lista, X_train):
    total = 0
    mantidas = 0
    mudaram = []

    for idx, (inst_idx, instancia) in enumerate(X_test.iterrows()):
        features_usadas = features_usadas_lista[idx]
        pred_original = modelo.predict(instancia.to_frame().T)

        perturbada = perturbar_instancia(instancia, features_usadas, X_train)
        pred_perturbada = modelo.predict(instancia.to_frame().T)

        total += 1
        if pred_perturbada == pred_original:
            mantidas += 1
        else:
            mudaram.append(idx)
            print(f"Instância {idx}: Predição mudou após perturbação → {pred_original} ➜ {pred_perturbada}")

    print(f"\n Robustez geral: {mantidas}/{total} ({(mantidas / total)*100:.2f}%) instâncias mantiveram a predição.")
    if mudaram:
        print(f"\nInstâncias que falharam no teste de robustez: {mudaram}")

#  CHAME A FUNÇÃO AQUI:
testar_robustez_pi(modelo, X_test, explicacoes, features_usadas_lista, X_train)



 Robustez geral: 154/154 (100.00%) instâncias mantiveram a predição.


In [12]:
from sklearn.linear_model import LogisticRegression

print("VALIDAÇÃO DAS PI-EXPLICAÇÕES\n" + "="*40)

for i, x in enumerate(X_test):
    pi_info = pi_explanations[i]
    pi_indices = set(pi_info["feature_indices"])
    original_pred = model.predict([x])[0]

    perturbed_x = x.copy()
    perturbations = {}

    for j in range(len(x)):
        if j not in pi_indices:
            original_value = x[j]
            new_value = np.random.choice(X_train[:, j])  # você pode trocar isso por outra estratégia
            perturbed_x[j] = new_value
            perturbations[j] = (original_value, new_value)

    new_pred = model.predict([perturbed_x])[0]

    print(f"\nInstância {i} | Real: {label_names[y_test[i]]} | Predita: {label_names[original_pred]}")
    print("PI-Explicação (essenciais):")
    for j in pi_indices:
        print(f"  - {feature_names[j]} = {x[j]:.2f}")

    if perturbations:
        print("Perturbações nas outras features:")
        for j, (orig, pert) in perturbations.items():
            print(f"  - {feature_names[j]}: original = {orig:.2f} → perturbação = {pert:.2f}")
    else:
        print("Nenhuma feature foi perturbada.")

    print(f"Predição após perturbação: {label_names[new_pred]} {'✅' if new_pred == original_pred else '❌'}")

print("\n✅ Validação completa.")


VALIDAÇÃO DAS PI-EXPLICAÇÕES


NameError: name 'pi_explanations' is not defined

In [16]:
def testar_robustez_visual(modelo, X_test, y_test, explicacoes, features_usadas_lista, X_train, class_names):
    for idx, (inst_idx, original) in enumerate(X_test.iterrows()):
        features_usadas = features_usadas_lista[idx]
        real_class = y_test[idx]
        pred_original = modelo.predict(original.to_frame().T)[0]

        perturbada = original.copy()
        diffs = {}
        for f in original.index:
            if f not in features_usadas:
                novo_valor = np.random.choice(X_train[f])
                if novo_valor != original[f]:
                    diffs[f] = (original[f], novo_valor)
                perturbada[f] = novo_valor

        pred_perturbada = modelo.predict(perturbada.to_frame().T)[0]
        status = "✅" if pred_perturbada == pred_original else "❌"

        print(f"\n📌 Instância {idx} | Real: {class_names[real_class]} | Predita: {class_names[pred_original]}")
        print(f"🔍 PI-explicação (features essenciais):")
        if features_usadas:
            for f in features_usadas:
                print(f"  - {f}: {original[f]:.3f}")
        else:
            print("  - Nenhuma")

        if diffs:
            print(f"\n🎲 Perturbações aplicadas nas outras features:")
            for f, (antes, depois) in diffs.items():
                print(f"  - {f}: {antes:.3f} → {depois:.3f}")
        else:
            print("\n🎲 Nenhuma feature foi perturbada.")

        print(f"\n🔄 Predição após perturbação: {class_names[pred_perturbada]} {status}")
        print("-"*60)



In [17]:
testar_robustez_visual(modelo, X_test, y_test, explicacoes, features_usadas_lista, X_train, class_names)


📌 Instância 0 | Real: Não Diabético | Predita: Não Diabético
🔍 PI-explicação (features essenciais):
  - glicose: 98.000
  - imc: 34.000
  - pressao_sangue: 58.000
  - idade: 43.000
  - diabetes_pedigree: 0.430
  - num_gravidezes: 6.000
  - insulina: 190.000
  - espessura_pele: 33.000

🎲 Nenhuma feature foi perturbada.

🔄 Predição após perturbação: Não Diabético ✅
------------------------------------------------------------

📌 Instância 1 | Real: Não Diabético | Predita: Não Diabético
🔍 PI-explicação (features essenciais):
  - glicose: 112.000
  - imc: 35.700
  - idade: 21.000
  - pressao_sangue: 75.000
  - diabetes_pedigree: 0.148
  - num_gravidezes: 2.000
  - espessura_pele: 32.000
  - insulina: 0.000

🎲 Nenhuma feature foi perturbada.

🔄 Predição após perturbação: Não Diabético ✅
------------------------------------------------------------

📌 Instância 2 | Real: Não Diabético | Predita: Não Diabético
🔍 PI-explicação (features essenciais):
  - glicose: 108.000
  - imc: 30.800
  - ida